In [ ]:




#导入torch，用于构建神经网络

import torch
from torch import nn


import numpy as np
import pandas as pd

#记录时间
from datetime import datetime

import time


import matplotlib.pyplot as plt

from scipy.interpolate import griddata
import scipy.io


#用于重复独立实验的随机种子
import random


#获取当前文件的名称

import os





In [ ]:


# %%

#设置随机种子
def setup_seed(seed):
    torch.manual_seed(seed)
    np.random.seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
        torch.backends.cuda.matmul.allow_tf32 = False# 尝试禁用这两个数据类型
        torch.backends.cudnn.allow_tf32 = False
        #torch.backends.cudnn.deterministic = True
        #torch.backends.cudnn.benchmark = False

        



        


In [ ]:
def sample(min,max, N):

    x = min + (max - min) * torch.rand((N, 1), dtype = datadtype).to(device).requires_grad_(True)
   
    return x




In [ ]:
class Net(nn.Module):
    def __init__(self, zysdtype,layers):
        super().__init__()#将父类的方法继承过来

        self.activation = nn.Tanh#选择tanh函数作为激活函数
        #nn.tanh是一个类，nn.Tanh（）是一个函数

        self.dtype = zysdtype

       
        self.depth = len(layers) - 1
        

        self.layers = nn.Sequential()

        for i in range(self.depth -1):
            self.layers.add_module('layer_%d' %i,
                                   nn.Linear(layers[i], layers[i+1],dtype = self.dtype))
            
            self.layers.add_module('activation_%d'%i,self.activation())

        #最后一层不通过激活函数
        self.layers.add_module("layer_%d" %(self.depth -1),nn.Linear(layers[-2], layers[-1],dtype = self.dtype))



      

    def forward(self, t,x):
        # #对输入进行归一化
        # t = 2.0*(t - t_lb)/(t_ub - t_lb) - 1.0
        # x = 2.0*(x - x_min)/(x_max - x_min) - 1.0
        H = torch.cat([t, x], 1)
        return self.layers(H)


In [ ]:


# %%

class HCPINN(nn.Module):
    def __init__(self,mydtype, layers,  t_min,t_max):
        
        super(HCPINN, self).__init__()
        self.dtype = mydtype
        self.layers = layers

        self.t_min = t_min
        self.t_max = t_max



   
        #采点
        self.N_i = 2*1200
        self.N_b = 2*int(1200 * (self.t_max - self.t_min))
        self.eta = 0.6
        self.N_r = 2*int(1e4 * (self.t_max - self.t_min))
        self.N_n = int(self.N_r * self.eta )
        
        self.t_ics_batch =  sample(self.t_min,self.t_min, self.N_i)
        self.x_ics_batch =  sample(x_min,x_max, self.N_i)
       

        
        self.t_bc1_batch =  sample(self.t_min,self.t_max, self.N_b)
        self.x_bc1_batch =  sample(x_min,x_min, self.N_b)
        


        
        self.x_bc2_batch =  sample(x_max,x_max, self.N_b)
        


        #Fetch residual mini-bathes
        
        self.t_res_batch =  sample(self.t_min,self.t_max, self.N_r)
        self.x_res_batch =  sample(x_min,x_max, self.N_r)
       


        self.u_ics_batch = initial(self.x_ics_batch)



        #定义优化器
        self.net = Net(self.dtype, self.layers).to(device)
        self.iter = 0
        self.adam = torch.optim.Adam(self.parameters(), lr = 1e-3)
        self.scheduler = torch.optim.lr_scheduler.StepLR(self.adam, step_size = 1000, gamma= 0.9)
        self.optimizer = torch.optim.LBFGS(
            self.parameters(),
            lr = 1.0,
            max_iter = 10000,
            max_eval = 50000,
            history_size = 200,
            tolerance_grad = 1e-12,
            tolerance_change = 1.0 * np.finfo(float).eps,
            line_search_fn = 'strong_wolfe'
        )




    
  
    
    def net_u(self, t,x):
        u = self.net(t,x)
        return u
    
    def net_r(self, t, x):

     
        u = self.net_u(t,x)
       

        u_t = torch.autograd.grad(u, t,
                                  grad_outputs=torch.ones_like(u),
                                  create_graph= True,
                                  retain_graph=True)[0]
        
        u_x = torch.autograd.grad(u, x,
                                  grad_outputs=torch.ones_like(u),
                                  create_graph= True,
                                  retain_graph=True)[0]

        u_xx = torch.autograd.grad(u_x, x,
                                  grad_outputs=torch.ones_like(u_x),
                                  create_graph= True,
                                  retain_graph=True)[0]
        
        u_xxx = torch.autograd.grad(u_xx, x,
                                  grad_outputs=torch.ones_like(u_xx),
                                  create_graph= True,
                                  retain_graph=True)[0]
        
      

        residual = u_t + u * u_x + 0.0025*u_xxx                   
        return residual
    
    def net_b(self, t, x):

     
        u = self.net_u(t,x)
        u_x = torch.autograd.grad(u, x,
                                  grad_outputs=torch.ones_like(u),
                                  create_graph= True,
                                  retain_graph=True)[0]
        

        return u_x


    
    def closure(self):
        self.adam.zero_grad()
        self.optimizer.zero_grad()



        #Evaluate predictions
        
      
        self.u_ics_pred = self.net_u(self.t_ics_batch, self.x_ics_batch)
        self.u_bc1_pred = self.net_u(self.t_bc1_batch, self.x_bc1_batch)
        self.u_bc2_pred = self.net_u(self.t_bc1_batch, self.x_bc2_batch)
        self.ux_bc1_pred = self.net_b(self.t_bc1_batch, self.x_bc1_batch)
        self.ux_bc2_pred = self.net_b(self.t_bc1_batch, self.x_bc2_batch)

     
        self.r_pred = self.net_r(self.t_res_batch, self.x_res_batch)

        #Boundary loss and Initial loss
        self.loss_ic = torch.mean((self.u_ics_pred- self.u_ics_batch)**2 )

        self.loss_ics = self.loss_ic


        self.loss_bc1 = torch.mean((self.u_bc1_pred - self.u_bc2_pred)**2 )
        self.loss_bc2 = torch.mean((self.ux_bc1_pred -self.ux_bc2_pred)**2 )

        self.loss_bcs = self.loss_bc2 + self.loss_bc1
        

        #Residual loss
        self.loss_res = torch.mean((self.r_pred)**2 )

        #Total loss
        self.loss =  self.loss_ics + self.loss_bcs + self.loss_res 

        self.loss.backward(retain_graph=True)
        

        self.iter += 1

        



        if self.iter % 1000 == 0:
            print(f'\r{self.iter},loss:{self.loss.item():.5e},i:{self.loss_ics.item():.2e},f:{self.loss_res.item():.2e},b:{self.loss_bcs.item():.2e}')
            if self.iter < nIter:
            
                # 生成随机索引
                shuffle_indices = torch.randperm(self.N_r, device = device)

                x_new_train = sample(x_min,x_max, self.N_n)

                # 打乱张量顺序
                self.x_res_batch = self.x_res_batch[shuffle_indices]

                self.x_res_batch[0:self.N_n] = x_new_train

                t_new_train = sample(self.t_min,self.t_max, self.N_n)

                # 打乱张量顺序
                self.t_res_batch = self.t_res_batch[shuffle_indices]

                self.t_res_batch[0:self.N_n] = t_new_train




        return self.loss
    
    def predict_u(self,X):
        t = X[:,0:1]
        x = X[:,1:2]
        #模型进入评估状态
        self.eval()
        with torch.no_grad():
            u = self.net_u(t,x).detach()
 
        return u
    




In [ ]:

# %%


class HCPINN_next(nn.Module):
    def __init__(self,mydtype, layers,t_min,t_max, premodel):
        
        super(HCPINN_next, self).__init__()
        

        self.premodel = premodel
        self.pre_net =  self.premodel.net
        self.dtype = mydtype

        self.t_min = t_min
        self.t_max = t_max

        #停止第一个网络参数的训练
        for param in self.premodel.parameters():
            param.requires_grad = False

      

 

        #Initialize network weights and biases
        self.layers = layers
      

        

        
        
        #采点
        self.N_i = 100
        self.N_b = int(1200 * (self.t_max - self.t_min))
        self.N_r = int(1e4 * (self.t_max - self.t_min))
        self.eta = 0.6
        self.N_n = int(self.eta * self.N_r)


       
        self.t_ics_batch =  sample(self.t_min,self.t_min, self.N_i)
        self.x_ics_batch =  sample(x_min,x_max, self.N_i)
        

        
        self.t_bc1_batch =  sample(self.t_min,self.t_max, self.N_b)
        self.x_bc1_batch =  sample(x_min,x_min, self.N_b)
       

        self.x_bc2_batch =  sample(x_max,x_max, self.N_b)
        

        #Fetch residual mini-bathes
        
        self.t_res_batch =  sample(self.t_min,self.t_max, self.N_r)
        self.x_res_batch =  sample(x_min,x_max, self.N_r)
        


        
        self.u_ics_batch = self.premodel.net_u(self.t_ics_batch, self.x_ics_batch)


   


        #将分段函数加入训练
        #定义第一个控制函数
        self.param_log = []
      
    
        self.w1 =  nn.Parameter(self.t_min + (self.t_max-self.t_min) * 0.5)
        

        print("p的初始化%.3e "%(self.w1.item()))


        #定义优化器
        self.iter = 0
        self.net = Net(self.dtype, self.layers).to(device)

        self.adam = torch.optim.Adam(self.parameters(), lr = 1e-3)
        self.scheduler = torch.optim.lr_scheduler.StepLR(self.adam, step_size= 1000, gamma= 0.9)
        self.optimizer = torch.optim.LBFGS(
            self.parameters(),#间断点在LBFGS的阶段也训练
            lr = 1.0,
            max_iter = 10000,
            max_eval = 50000,
            history_size = 200,
            tolerance_grad = 1e-12,
            tolerance_change = 1.0 * np.finfo(float).eps,
            line_search_fn = 'strong_wolfe'
        )




    
    def impact_func(self,t):
  
 
        condition1 = t <= self.t_min

        if self.w1 < self.t_max:
            t_right = self.w1
        else:
            t_right = self.t_max
      
        condition2 = t <= t_right
       

     
        B = (t - self.t_min) / (t_right - self.t_min)


        result = torch.where(condition1, torch.zeros_like(t), torch.where(condition2, -2 * B**3 + 3 * B**2, torch.ones_like(t) ) )
                                   
        return result
    
  
    



    def net_u(self, t,x):
        ww = self.impact_func(t)
        u =  (1-ww)*self.pre_net(t,x) + ww* self.net(t,x)
        return u
    
  
    
    def net_r(self, t, x):

        u = self.net_u(t,x)

        u_t = torch.autograd.grad(u, t,
                                  grad_outputs=torch.ones_like(u),
                                  create_graph= True,
                                  retain_graph=True)[0]
        
        u_x = torch.autograd.grad(u, x,
                                  grad_outputs=torch.ones_like(u),
                                  create_graph= True,
                                  retain_graph=True)[0]

        u_xx = torch.autograd.grad(u_x, x,
                                  grad_outputs=torch.ones_like(u_x),
                                  create_graph= True,
                                  retain_graph=True)[0]
        
        u_xxx = torch.autograd.grad(u_xx, x,
                                  grad_outputs=torch.ones_like(u_xx),
                                  create_graph= True,
                                  retain_graph=True)[0]
        
      

        residual = u_t + u * u_x + 0.0025*u_xxx  

                             
        return residual
    
    def net_b(self, t, x):

        u = self.net_u(t,x)

        
        u_x = torch.autograd.grad(u, x,
                                  grad_outputs=torch.ones_like(u),
                                  create_graph= True,
                                  retain_graph=True)[0]
        
       
        return u_x


    
    def closure(self):
        self.adam.zero_grad()
        self.optimizer.zero_grad()

    
        #Evaluate predictions
        
       

        self.u_ics_pred = self.net_u(self.t_ics_batch, self.x_ics_batch)
        self.u_bc1_pred = self.net_u(self.t_bc1_batch, self.x_bc1_batch)
        self.u_bc2_pred = self.net_u(self.t_bc1_batch, self.x_bc2_batch)
        self.ux_bc1_pred = self.net_b(self.t_bc1_batch, self.x_bc1_batch)
        self.ux_bc2_pred = self.net_b(self.t_bc1_batch, self.x_bc2_batch)

     
        self.r_pred = self.net_r(self.t_res_batch, self.x_res_batch)

        #Boundary loss and Initial loss
        self.loss_ic = torch.mean((self.u_ics_pred- self.u_ics_batch)**2 )
        self.loss_bc1 = torch.mean((self.u_bc1_pred - self.u_bc2_pred)**2 )
        self.loss_bc2 = torch.mean((self.ux_bc1_pred -self.ux_bc2_pred)**2 )

        self.loss_bcs = self.loss_bc2 + self.loss_bc1
        self.loss_ics = self.loss_ic

        #Residual loss
        self.loss_res = torch.mean((self.r_pred)**2 )

        #Total loss
        self.loss = self.loss_bcs + self.loss_res

        self.loss.backward(retain_graph=True)
        

        self.iter += 1

        
        
        self.param_log.append([self.w1.item()])
 

        if self.iter % 1000 == 0:
            print(f'\r{self.iter},loss:{self.loss.item():.5e},i:{self.loss_ics.item():.2e},f:{self.loss_res.item():.2e},b:{self.loss_bcs.item():.2e},p:{self.w1.item():.2e}')
           
            
            if self.iter < nIter :
                # 生成随机索引
            
                shuffle_indices = torch.randperm(self.N_r, device = device)

                x_new_train = sample(x_min,x_max, self.N_n)

                

                # 打乱张量顺序
                self.x_res_batch = self.x_res_batch[shuffle_indices]

                self.x_res_batch[0:self.N_n] = x_new_train

                t_new_train = sample(self.t_min,self.t_max,self.N_n)

                # 打乱张量顺序
                self.t_res_batch = self.t_res_batch[shuffle_indices]

                self.t_res_batch[0:self.N_n] = t_new_train



        return self.loss
    
    def predict_u(self,X):
        t = X[:,0:1]
        x = X[:,1:2]
        self.eval()
        with torch.no_grad():
            u = self.net_u(t,x).detach()
            
        return u
    




In [ ]:



# %%

# %%
#Define initial condition 

def initial(x):

   
    u = torch.cos(torch.pi * x)
    return u



In [ ]:


# %%

# %%

# %%
#画出区间的图像以及求误差
def plot_error_block(model,t_min,t_max):
   
    n_min = int(t_min * num_points)
    n_max = int(t_max * num_points)
    #n_test = n_max - n_min

    t_block = t_total[n_min:n_max +1]
    T_block, X_block = torch.meshgrid(t_block, x_total, indexing='ij')

    #组装
 
    X_star_block = torch.cat((T_block.flatten()[:, None], X_block.flatten()[:,None]), dim=1)

    #引入真实的信息
    u0_star = data[:,n_min:n_max +1].T
    u_star = u0_star.flatten()[:,None]

    #网络预测的结果
    u_pred_block = model.predict_u(X_star_block)

    #计算误差
    error_u =  torch.norm(u_star - u_pred_block, p=2) /torch.norm(u_star, p=2)
    error_u1 = torch.norm(u_star - u_pred_block, p=1) / len(X_star_block)
    error_uinf = torch.norm(u_star - u_pred_block, p=float('inf'))
    
    print("\n查看区间：[%.2f,%.2f]"%(T_block.min().item(), T_block.max().item()))
    print("\nRelative L2 error_u: {:.5e}".format(error_u))
    print("\nRelative L1 error_u: {:.5e}".format(error_u1))
    print("\nRelative Linf error_u: {:.5e}".format(error_uinf))
    


    return u_pred_block, error_u.item(), error_u1.item(), error_uinf.item()



In [ ]:
def plot_func(t, func1,p_log):
    fig = plt.figure(figsize=(15, 3),dpi=200)
    plt.subplot(1, 2, 1)
    plt.plot(t.cpu().detach().numpy(), func1(t).cpu().detach().numpy())
    plt.title("eta(t)")


    plt.subplot(1, 2, 2)
    plt.plot(range(0, len(p_log)), p_log)
    plt.title("p_log")
    plt.show()
   

In [ ]:


def train_block(t_min, t_max, num_block,train_num):
    train_time = 0
    models = []
    error_total = []
   
    t_block = torch.linspace(t_min, t_max, num_block + 1).to(device)

    train_time_total = []
    for i in range(num_block):
        
        print('训练区间：[%.2f,%.2f]'%(t_block[i], t_block[i+1]))

        
        if i == 0:
            
            model = HCPINN(datadtype,layers, t_block[i],t_block[i+1])

                #进行训练
            start_time = time.time()
            print('\n\t====Begin Adam====')
            for adam_iter in range(nIter):
                    model.closure()
                    model.adam.step()
                    model.scheduler.step()
            print('\n\t====Begin LBFSG====')
            model.optimizer.step(model.closure)
            block_time =  (time.time() - start_time)/60
            
          


        else:

            model = HCPINN_next(datadtype, layers,t_block[i], t_block[i+1],   models[-1])



            #进行训练
            start_time = time.time()
            print('\n\t====Begin Adam====')
            for adam_iter in range(nIter):
                    model.closure()
                    model.adam.step()
                    model.scheduler.step()
            print('\n\t====Begin LBFSG====')
            model.optimizer.step(model.closure)
            block_time =  (time.time() - start_time)/60

            #画出铰链函数
            plot_func(torch.linspace(t_min, t_max + 0.1, 10000).to(device), model.impact_func,model.param_log)
            
           

            
            #保存当前参数的变化
            df_param_log = pd.DataFrame(model.param_log, columns=['p'])

         

            # 将DataFrame保存为Excel文件
            df_param_log.to_excel(folder_path + "/第%d次训练_[%.2f,%.2f]_p_log.xlsx"%(train_num, t_block[i],t_block[i+1]), index=False)

     

          
        
        train_time_total.append(block_time)
        print('区间[%.2f,%.2f]的训练时间：%.2f min(s)'%(t_block[i],t_block[i+1] ,(time.time() - start_time) /60)) 
        print("Final loss: %.5e,Total iteration: %d"%(model.loss.item(),model.iter))

        
        
        #保存网络
        torch.save(model.state_dict(),folder_path + "/第%d次训练_[%.2f,%.2f].pth"%(train_num, t_block[i],t_block[i+1]))

        _, *error_block= plot_error_block(model,t_block[i], t_block[i +1])
  

        #记录实验结果
        models.append(model)
        error_total.append(error_block)
    train_time = sum(train_time_total)
    train_time_total.append(train_time)
    print('\n#小区间个数：%d，总训练时长:%.2f  min(s)'%(num_block,train_time))

    return models,error_total, train_time_total



# %%


In [ ]:



# %%
def u_pred_whole(models, t_min, t_max,num_block,train_times_num):

    
    t_block = torch.linspace(t_min, t_max, num_block + 1).to(device)


    
    n_min = int(t_min * num_points)
    n_max = int(t_max * num_points)
    n_mid = n_max - n_min

    t_total_now = t_total[n_min:n_max +1]



    
    T_total_now,X_total_now = torch.meshgrid(t_total_now,x_total, indexing='ij')

    X_star_total_now = torch.cat((T_total_now.flatten()[:,None],X_total_now.flatten()[:,None]), dim=1)

    u0_star =  data[:,n_min:n_max +1].T
    u_star = u0_star.flatten()[:,None]

    u_combine = []

    for block_index in range(num_block):
        #对每个小区间进行更加详细的计算
        n_min_now = int(t_block[block_index] * num_points)
        n_max_now = int(t_block[block_index +1] * num_points)

        if block_index ==0:
            
            tt = t_total_now[n_min_now : n_max_now+1]

        else:
            tt = t_total_now[n_min_now+1 :  n_max_now+1]


        T_pred_now,X_pred_now = torch.meshgrid(tt,x_total, indexing='ij')

        XX_star_pred = torch.cat((T_pred_now.flatten()[:,None],X_pred_now.flatten()[:,None]), dim=1)
      

        u_pred = models[block_index].predict_u(XX_star_pred)
      
        u_combine.append(u_pred)
 
    
    u_com = torch.cat(u_combine, dim=0)



    erroru = torch.norm(u_star - u_com, p=2) / torch.norm(u_star, p=2)
    erroru1 = torch.norm(u_star - u_com,p=1)/len(X_star_total_now)
    erroruinf = torch.norm(u_star - u_com, p=float('inf'))
    error_whole = [erroru.item(), erroru1.item(), erroruinf.item()]
    print('\n#查看区间[%.2f,%.2f]:'%(t_min, t_max))
    print('#error L2:{:.5e}'.format(erroru))
    print('#error L1:{:.5e}'.format(erroru1))
    print('#error Linf:{:.5e}'.format(erroruinf))

    U_pred = griddata(X_star_total_now.cpu(), u_com.cpu().flatten(), (T_total_now.cpu(),X_total_now.cpu()),method = 'cubic')
    U_star = griddata(X_star_total_now.cpu(), u0_star.cpu().flatten(), (T_total_now.cpu(), X_total_now.cpu()), method='cubic')
   
   
    #画出全区间的图像
    fig = plt.figure(figsize=(18, 5),dpi=200)
    plt.subplot(1, 3, 1)
    plt.pcolor(T_total_now.cpu(), X_total_now.cpu(), U_star, cmap='jet')
    plt.colorbar()
    plt.xlabel('$t$')
    plt.ylabel('$x$')
    plt.title(r'Reference solution')
    plt.tight_layout()

    plt.subplot(1, 3, 2)
    plt.pcolor(T_total_now.cpu(), X_total_now.cpu(), U_pred, cmap='jet')
    plt.colorbar()
    plt.xlabel('$t$')
    plt.ylabel('$x$')
    plt.title(r'Predicted solution')
    plt.tight_layout()

    plt.subplot(1, 3, 3)
    plt.pcolor(T_total_now.cpu(), X_total_now.cpu(), abs(U_star - U_pred), cmap='jet')
    plt.colorbar()
    plt.xlabel('$t$')
    plt.ylabel('$x$')
    plt.title('Point-wise Error')
    plt.tight_layout()
    plt.savefig(folder_path + '/区间个数%d_第%d次训练_U_pred.pdf'%(num_block,train_times_num))
    #plt.show()  
    plt.close()

    #画出固定t，u的snap
    fig = plt.figure(figsize=(18, 5),dpi=200)
    plt.subplot(1, 3, 1)
    plt.plot(x_total.cpu(), U_star[0,:], 'r-',linewidth = 2.5,label= 'Reference')
    plt.plot(x_total.cpu(), U_pred[0,:], 'b--',linewidth = 2.5,label= 'Prediction')
    plt.xlabel('$x$')
    plt.legend()
    plt.title('t = %.2f'%(t_total_now[0]))
    plt.tight_layout()

    plt.subplot(1, 3, 2)
    plt.plot(x_total.cpu(), U_star[int(n_mid/2),:], 'r-',linewidth = 2.5,label= 'Reference')
    plt.plot(x_total.cpu(), U_pred[int(n_mid/2),:], 'b--',linewidth = 2.5,label= 'Prediction')
    plt.xlabel('$x$')
    plt.legend()
    plt.title('t = %.2f'%(t_total_now[int(n_mid/2)]))
    plt.tight_layout()


    plt.subplot(1, 3, 3)
    plt.plot(x_total.cpu(), U_star[n_mid,:], 'r-',linewidth = 2.5,label= 'Reference')
    plt.plot(x_total.cpu(), U_pred[n_mid,:], 'b--',linewidth = 2.5,label= 'Prediction')
    plt.xlabel('$x$')
    plt.legend()
    plt.title('t = %.2f'%(t_total_now[n_mid]))
    plt.tight_layout()
    plt.savefig(folder_path + '/区间个数%d_第%d次训练_u_snap.pdf'%(num_block,train_times_num))
    #plt.show()
    plt.close()

    return u_com, error_whole




# %%


In [ ]:



#定义超参数
datadtype = torch.float32

#CUDA support
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#device = torch.device( "cpu")

print(f'Using {device} to train...')
print(f'The code is trained at {datetime.now()}')




notebook_name =  'result'
print(notebook_name)


In [ ]:

import shutil
# 定义文件夹路径
folder_path = notebook_name

# 如果文件夹存在，先删除
if os.path.exists(folder_path):
    
    shutil.rmtree(folder_path)
    print(f"文件夹 '{folder_path}' 已删除。")

# 重新创建文件夹
os.makedirs(folder_path)
print(f"文件夹 '{folder_path}' 已重新创建。")



In [ ]:


# %%

t_min, t_max = 0.0, 1
x_min, x_max = -1, 1



#导入数据
num_points = 200

data_source = scipy.io.loadmat("D:/02_master/课题/硬约束/实验代码/kdv_long.mat")


data = data_source['usol']
t_total = data_source['t'].flatten()[:,None]
x_total = data_source['x'].flatten()[:,None]
data = torch.tensor(data, dtype = datadtype).to(device)
t_total = torch.tensor(t_total, dtype = datadtype).to(device).squeeze()
x_total = torch.tensor(x_total, dtype = datadtype).to(device).squeeze()


#定义独立重复实验次数以及小区间个数


num_block_start = 4
num_block_end = 4
train_times = 5



num_neural = 50


layers = [2, num_neural, num_neural, num_neural,  1]


nIter = 5000




In [ ]:
def caculate_dist(model_big, model_small, t_min, t_max):
    n_min = int(t_min * num_points)
    n_max = int(t_max * num_points)

    t_predict = t_total[n_min : n_max +1]
    T_predict, X_block = torch.meshgrid(t_predict, x_total, indexing = 'ij')

    X_star_predict = torch.cat((T_predict.flatten()[:, None], X_block.flatten()[:,None]), dim=1)

    u_pred_big = model_big.predict_u(X_star_predict)
    u_pred_small = model_small.predict_u(X_star_predict)
    #计算误差
    dist =  torch.norm(u_pred_big - u_pred_small, p=2) /torch.norm(u_pred_small, p=2)

    print("\n两个模型在预测区间[%.4f,%.4f]上的L2相对误差为%.5e"%(T_predict.min().item(), T_predict.max().item(), dist))

    return dist.item()

In [ ]:
def delta_adaptivate(t_min, t_max, delta):
    train_time = 0
    train_time_log = []
    step_log = []
    dist_log = []
    block_log = []
    
    idea_step = (t_min + t_max)/2 #float(input("请输入："))
    
    step_log.append(idea_step)
    print("-------**确定步长**-------")
    print("\n查看步长 %.3f 是否可行"%(step_log[-1]))

    #看输入可否作为步长
    model_big = HCPINN(datadtype,layers,t_min, 2 * step_log[-1])
    block_log.append([t_min, 2 * step_log[-1]])
    model_small = HCPINN(datadtype,layers,t_min, step_log[-1])
    block_log.append([t_min, step_log[-1]])
    print("\n第%d次测试的区间为[%.4f, %.4f]和[%.4f, %.4f]"%(len(dist_log) +1 , t_min, 2*step_log[-1],t_min, step_log[-1]))

    print("\n训练[%.4f, %.4f]中..."%(t_min, 2*step_log[-1]))
    print('\n\t====Begin Adam====')
    start_time = time.time()
    for adam_iter in range(nIter):
          model_big.closure()
          model_big.adam.step()
          model_big.scheduler.step()
    print('\n\t====Begin LBFSG====')
    model_big.optimizer.step(model_big.closure)
    model1_time = (time.time() - start_time)/60
    print("\n训练[%.4f, %.4f]的时长为：%.2f min(s)"%(t_min, 2*step_log[-1], model1_time))
    train_time_log.append(model1_time)


    print("\n训练[%.4f, %.4f]中..."%(t_min, step_log[-1]))
    print('\n\t====Begin Adam====')
    start_time = time.time()
    for adam_iter in range(nIter):
        model_small.closure()
        model_small.adam.step()
        model_small.scheduler.step()
    print('\n\t====Begin LBFSG====')
    model_small.optimizer.step(model_small.closure)
    model1_time = (time.time() - start_time)/60
    print("\n训练[%.4f, %.4f]的时长为：%.2f min(s)"%(t_min, step_log[-1], model1_time))
    train_time_log.append(model1_time)


    #比较两个模型在小区间上的表现
    print("\n不同模型在预测区间[%.4f, %.4f]的表现如下："%(t_min, step_log[-1]))
    print("在[%.4f, %.4f]上训练的模型："%(t_min, 2*step_log[-1]))
    uu_big = plot_error_block(model_big,t_min, step_log[-1])
    print("在[%.4f, %.4f]上训练的模型："%(t_min, step_log[-1]))
    uu_small = plot_error_block(model_small,t_min, step_log[-1])



    dist = caculate_dist(model_big, model_small, t_min, step_log[-1])

    dist_log.append(dist)

    

        

    while len(dist_log) <= 3 : #还是在确定步长dist_log[-1] > delta
        #首先进一步缩小步长
        step_log.append(0.5 * step_log[-1])
        print("查看步长%.5e是否可行"%(step_log[-1]))

        model_big = model_small#HCPINN(datadtype,layers,t_min,  step_log[-1])  直接将上面的小区间继承过来
        model_small = HCPINN(datadtype,layers,t_min,step_log[-1])

        print("进一步缩小步长，第%d次测试的区间为[%.4f, %.4f]和[%.4f, %.4f]"%(len(dist_log)+1, t_min, 2*step_log[-1],t_min,  step_log[-1]))
        print("区间[%.4f, %.4f]的模型已在上一步完成"%( t_min, step_log[-1]))

        print("训练[%.4f, %.4f]中..."%(t_min,  step_log[-1]))
        print('\n\t====Begin Adam====')
        start_time = time.time()
        for adam_iter in range(nIter):
                model_small.closure()
                model_small.adam.step()
                model_small.scheduler.step()
        print('\n\t====Begin LBFSG====')
        model_small.optimizer.step(model_small.closure)
        model1_time = (time.time() - start_time)/60
        print("训练[%.4f, %.4f]的时长为：%.2f  min(s)"%(t_min,step_log[-1], model1_time))
        train_time_log.append(model1_time)


        #比较两个模型在小区间上的表现
        print("\n不同模型在预测区间[%.4f, %.4f]的表现如下："%(t_min,  step_log[-1]))
        print("在[%.4f, %.4f]上训练的模型："%(t_min, 2*step_log[-1]))
        uu_big = plot_error_block(model_big,t_min,step_log[-1])
        print("在[%.4f, %.4f]上训练的模型："%(t_min, step_log[-1]))
        uu_small = plot_error_block(model_small,t_min, step_log[-1])



        dist = caculate_dist(model_big, model_small, t_min, step_log[-1])

        dist_log.append(dist)

    #else:
    train_time = sum(train_time_log)
    
    print("总的训练时长为%.2f min(s), 步长为%.5e"%(train_time, step_log[-1]))
    return step_log,dist_log,train_time_log


In [ ]:
delta = 1e-3

In [ ]:
step_log,dist_log,train_time_log = delta_adaptivate(t_min, t_max,  delta)

In [ ]:
step_log,dist_log,train_time_log